In [1]:
import anndata as ad
import scvi
import scanpy as sc
import mrvi
import pandas as pd
import scipy as sp
import numpy as np
import pickle as pkl
from sklearn.cluster import KMeans
import igraph as ig
import leidenalg as la
import utils

Global seed set to 0
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
adata = sc.read('./../data/MrVIoutputs/bacdrop.h5ad')


In [3]:
ranked_genes = utils.mrvi_identify_cell_states(adata, 'X_mrvi_z', neighbor_method='knn')

/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'scores'] = scores[global_indices]
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:

In [4]:
# sc.pl.umap(adata, use_raw=True)
# sc.pl.embedding(adata, basis='X_mrvi_z')

In [5]:
df = ranked_genes.head(10)

In [6]:
# sc.pl.umap(adata, use_raw=False)
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,cds-WP_004174069.1-2,cds-WP_002920103.1,cds-WP_012068464.1,cds-WP_002919515.1,cds-WP_004174069.1,cds-WP_004174069.1,cds-WP_004174069.1-2,cds-WP_013815099.1-12,cds-WP_032429520.1,cds-WP_013815099.1-29,...,cds-WP_179139609.1,cds-WP_002888735.1,cds-WP_013815099.1-7,cds-WP_013815099.1-79,cds-WP_004145074.1,cds-WP_013815099.1-79,cds-WP_002919219.1,cds-WP_004174069.1,cds-WP_032428654.1,cds-WP_002884740.1
1,cds-WP_004174069.1,cds-WP_032429669.1,cds-WP_002898162.1,cds-WP_032424651.1,cds-WP_004174069.1-2,cds-WP_004174069.1-2,cds-WP_004174069.1,cds-WP_013815099.1-28,cds-AF52_RS26610,cds-WP_013815099.1-78,...,cds-WP_004164645.1,cds-WP_002920103.1,cds-WP_013815099.1-17,cds-WP_013815099.1-58,cds-WP_001181005.1,cds-WP_013815099.1-35,cds-WP_032430210.1,cds-WP_004174069.1-2,cds-WP_032424651.1,cds-WP_002888735.1
2,cds-WP_012068464.1,cds-WP_032424651.1,cds-WP_002888735.1,cds-WP_032429833.1,cds-WP_002884146.1,cds-WP_004150943.1,cds-WP_002888735.1,cds-WP_013815099.1-69,cds-WP_171290763.1,cds-WP_013815099.1-15,...,cds-AF52_RS27385,cds-WP_004901914.1,cds-WP_013815099.1-66,cds-WP_013815099.1-2,cds-WP_004151523.1,cds-WP_013815099.1-63,cds-WP_004149864.1,cds-WP_002922413.1,cds-WP_002888735.1,cds-WP_004885708.1
3,cds-WP_032429669.1,cds-WP_004174069.1-2,cds-WP_002920103.1,cds-WP_002919766.1,cds-WP_002888735.1,cds-WP_002920554.1,cds-WP_004183439.1,cds-WP_013815099.1-9,cds-AF52_RS27170,cds-WP_013815099.1-7,...,cds-AF52_RS26890,cds-WP_002898162.1,cds-WP_013815099.1-59,cds-WP_013815099.1-16,cds-WP_004146997.1,cds-WP_013815099.1-10,cds-WP_002903693.1,cds-WP_020802781.1,cds-WP_004174069.1-2,cds-WP_002919794.1
4,cds-WP_004144748.1,cds-WP_004174069.1,cds-WP_002919796.1,cds-WP_002888735.1,cds-WP_004901914.1,cds-WP_002919219.1,cds-WP_004179067.1,cds-WP_013815099.1-63,cds-AF52_RS27605,cds-WP_013815099.1-71,...,cds-AF52_RS27585,cds-WP_002888731.1,cds-WP_013815099.1-42,cds-WP_013815099.1-47,cds-WP_001207203.1,cds-WP_013815099.1-62,cds-WP_032429636.1,cds-WP_004144995.1,cds-WP_020802783.1,cds-WP_004174069.1
5,cds-WP_002918559.1,cds-WP_004179067.1,cds-WP_002898408.1,cds-WP_002919219.1,cds-WP_002919219.1,cds-WP_002882755.1,cds-WP_021441082.1,cds-WP_013815099.1-26,cds-WP_179139609.1,cds-WP_013815099.1-42,...,cds-AF52_RS27330,cds-WP_002914070.1,cds-WP_013815099.1-71,cds-WP_013815099.1-75,cds-WP_002885441.1,cds-WP_013815099.1-56,cds-WP_032429785.1,cds-WP_002884146.1,cds-WP_004212509.1,cds-WP_002887843.1
6,cds-WP_004177236.1,cds-WP_002919766.1,cds-WP_004150800.1,cds-WP_020802781.1,cds-WP_001207203.1,cds-WP_002918250.1,cds-WP_032429669.1,cds-WP_013815099.1-78,cds-AF52_RS27080,cds-WP_013815099.1-49,...,cds-AF52_RS27035,cds-WP_002889299.1,cds-WP_013815099.1-47,cds-WP_013815099.1,cds-WP_000829818.1,cds-WP_013815099.1-72,cds-WP_004151118.1,cds-WP_085921749.1,cds-WP_004151856.1,cds-WP_004151552.1
7,cds-WP_020802781.1,cds-WP_002898162.1,cds-WP_004144748.1,cds-WP_002919796.1,cds-WP_002884743.1,cds-WP_002919748.1,cds-WP_002884747.1,cds-WP_013815099.1-29,cds-WP_170999280.1,cds-WP_013815099.1-3,...,cds-WP_170999280.1,cds-WP_032429671.1,cds-WP_013815099.1-43,cds-WP_013815099.1-27,cds-WP_002884036.1,cds-WP_187079192.1,cds-WP_002898303.1,cds-WP_004152310.1,cds-WP_004191727.1,cds-WP_002891635.1
8,cds-WP_002888735.1,cds-WP_020802781.1,cds-WP_004900541.1,cds-WP_002914147.1,cds-WP_002884747.1,cds-WP_002919224.1,cds-WP_004144748.1,cds-WP_013815099.1-31,cds-AF52_RS26890,cds-WP_050484282.1,...,cds-WP_013815099.1-54,cds-WP_002919667.1,cds-WP_013815099.1-26,cds-WP_013815099.1-83,cds-WP_002922941.1,cds-WP_013815099.1-71,cds-WP_002915934.1,cds-WP_002898422.1,cds-WP_004148352.1,cds-WP_002920113.1
9,cds-WP_002920103.1,cds-WP_004144997.1,cds-WP_002891635.1,cds-WP_002898162.1,cds-WP_002919786.1,cds-WP_002919665.1,cds-WP_021441395.1,cds-WP_013815099.1-15,cds-WP_187079182.1,cds-WP_013815099.1-31,...,cds-WP_171290763.1,cds-WP_002919794.1,cds-WP_013815099.1-53,cds-WP_013815099.1-81,cds-WP_002910103.1,cds-WP_032430171.1,cds-WP_004185139.1,cds-WP_00415

In [7]:
df.to_csv("./../out/test.csv")
# sc.pl.embedding(adata, basis='X_mrvi_u')

In [8]:
df_test = pd.read_csv("./../out/test.csv")

In [9]:
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,cds-WP_004174069.1-2,cds-WP_002920103.1,cds-WP_012068464.1,cds-WP_002919515.1,cds-WP_004174069.1,cds-WP_004174069.1,cds-WP_004174069.1-2,cds-WP_013815099.1-12,cds-WP_032429520.1,cds-WP_013815099.1-29,...,cds-WP_179139609.1,cds-WP_002888735.1,cds-WP_013815099.1-7,cds-WP_013815099.1-79,cds-WP_004145074.1,cds-WP_013815099.1-79,cds-WP_002919219.1,cds-WP_004174069.1,cds-WP_032428654.1,cds-WP_002884740.1
1,cds-WP_004174069.1,cds-WP_032429669.1,cds-WP_002898162.1,cds-WP_032424651.1,cds-WP_004174069.1-2,cds-WP_004174069.1-2,cds-WP_004174069.1,cds-WP_013815099.1-28,cds-AF52_RS26610,cds-WP_013815099.1-78,...,cds-WP_004164645.1,cds-WP_002920103.1,cds-WP_013815099.1-17,cds-WP_013815099.1-58,cds-WP_001181005.1,cds-WP_013815099.1-35,cds-WP_032430210.1,cds-WP_004174069.1-2,cds-WP_032424651.1,cds-WP_002888735.1
2,cds-WP_012068464.1,cds-WP_032424651.1,cds-WP_002888735.1,cds-WP_032429833.1,cds-WP_002884146.1,cds-WP_004150943.1,cds-WP_002888735.1,cds-WP_013815099.1-69,cds-WP_171290763.1,cds-WP_013815099.1-15,...,cds-AF52_RS27385,cds-WP_004901914.1,cds-WP_013815099.1-66,cds-WP_013815099.1-2,cds-WP_004151523.1,cds-WP_013815099.1-63,cds-WP_004149864.1,cds-WP_002922413.1,cds-WP_002888735.1,cds-WP_004885708.1
3,cds-WP_032429669.1,cds-WP_004174069.1-2,cds-WP_002920103.1,cds-WP_002919766.1,cds-WP_002888735.1,cds-WP_002920554.1,cds-WP_004183439.1,cds-WP_013815099.1-9,cds-AF52_RS27170,cds-WP_013815099.1-7,...,cds-AF52_RS26890,cds-WP_002898162.1,cds-WP_013815099.1-59,cds-WP_013815099.1-16,cds-WP_004146997.1,cds-WP_013815099.1-10,cds-WP_002903693.1,cds-WP_020802781.1,cds-WP_004174069.1-2,cds-WP_002919794.1
4,cds-WP_004144748.1,cds-WP_004174069.1,cds-WP_002919796.1,cds-WP_002888735.1,cds-WP_004901914.1,cds-WP_002919219.1,cds-WP_004179067.1,cds-WP_013815099.1-63,cds-AF52_RS27605,cds-WP_013815099.1-71,...,cds-AF52_RS27585,cds-WP_002888731.1,cds-WP_013815099.1-42,cds-WP_013815099.1-47,cds-WP_001207203.1,cds-WP_013815099.1-62,cds-WP_032429636.1,cds-WP_004144995.1,cds-WP_020802783.1,cds-WP_004174069.1
5,cds-WP_002918559.1,cds-WP_004179067.1,cds-WP_002898408.1,cds-WP_002919219.1,cds-WP_002919219.1,cds-WP_002882755.1,cds-WP_021441082.1,cds-WP_013815099.1-26,cds-WP_179139609.1,cds-WP_013815099.1-42,...,cds-AF52_RS27330,cds-WP_002914070.1,cds-WP_013815099.1-71,cds-WP_013815099.1-75,cds-WP_002885441.1,cds-WP_013815099.1-56,cds-WP_032429785.1,cds-WP_002884146.1,cds-WP_004212509.1,cds-WP_002887843.1
6,cds-WP_004177236.1,cds-WP_002919766.1,cds-WP_004150800.1,cds-WP_020802781.1,cds-WP_001207203.1,cds-WP_002918250.1,cds-WP_032429669.1,cds-WP_013815099.1-78,cds-AF52_RS27080,cds-WP_013815099.1-49,...,cds-AF52_RS27035,cds-WP_002889299.1,cds-WP_013815099.1-47,cds-WP_013815099.1,cds-WP_000829818.1,cds-WP_013815099.1-72,cds-WP_004151118.1,cds-WP_085921749.1,cds-WP_004151856.1,cds-WP_004151552.1
7,cds-WP_020802781.1,cds-WP_002898162.1,cds-WP_004144748.1,cds-WP_002919796.1,cds-WP_002884743.1,cds-WP_002919748.1,cds-WP_002884747.1,cds-WP_013815099.1-29,cds-WP_170999280.1,cds-WP_013815099.1-3,...,cds-WP_170999280.1,cds-WP_032429671.1,cds-WP_013815099.1-43,cds-WP_013815099.1-27,cds-WP_002884036.1,cds-WP_187079192.1,cds-WP_002898303.1,cds-WP_004152310.1,cds-WP_004191727.1,cds-WP_002891635.1
8,cds-WP_002888735.1,cds-WP_020802781.1,cds-WP_004900541.1,cds-WP_002914147.1,cds-WP_002884747.1,cds-WP_002919224.1,cds-WP_004144748.1,cds-WP_013815099.1-31,cds-AF52_RS26890,cds-WP_050484282.1,...,cds-WP_013815099.1-54,cds-WP_002919667.1,cds-WP_013815099.1-26,cds-WP_013815099.1-83,cds-WP_002922941.1,cds-WP_013815099.1-71,cds-WP_002915934.1,cds-WP_002898422.1,cds-WP_004148352.1,cds-WP_002920113.1
9,cds-WP_002920103.1,cds-WP_004144997.1,cds-WP_002891635.1,cds-WP_002898162.1,cds-WP_002919786.1,cds-WP_002919665.1,cds-WP_021441395.1,cds-WP_013815099.1-15,cds-WP_187079182.1,cds-WP_013815099.1-31,...,cds-WP_171290763.1,cds-WP_002919794.1,cds-WP_013815099.1-53,cds-WP_013815099.1-81,cds-WP_002910103.1,cds-WP_032430171.1,cds-WP_004185139.1,cds-WP_00415

In [16]:
for i in range(len(df_test.columns)):
    print(len(np.unique(df_test['0'].to_numpy())))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


"Multiple values can be given to color. In the following example we will plot 6 genes: ‘CD79A’, ‘MS4A1’, ‘IGJ’, CD3D’, ‘FCER1A’, and ‘FCGR3A’ to get an idea on where those marker genes are being expressed."

In [11]:
# TODO: figure out the markers for bacdrop and plot adding them as the colour label to identify where they're being expressed 
# e.g. sc.pl.umap(pbmc, color=['CD79A', 'MS4A1', 'IGJ', 'CD3D', 'FCER1A', 'FCGR3A', 'n_counts', 'bulk_labels'])